# 名義尺度の変数をダミー変数に変換する方法を知る(2022/03/19)
---

## 概要
---
本notebookではダミー変数の概要と，ダミー変数変換のPythonを用いた実装を取り扱う．

データセットには「長さ」「重さ」などと言った量的変数の他に「色」「サイズ」などといった質的変数も入りうる．こうした質的変数を計算機で取り扱うために，間接的な数値へ変換する手法が存在する．ダミー変数もその一種である．

## ダミー変数
---

ダミー変数は，名義尺度(名義特徴量ともいう)の変数を計算機で取り扱えるように$\{0, 1\}$の二値に変換したものである．

通常名義尺度と呼ばれるような変数は，「男」「女」や「赤」「青」「黄色」というように，それぞれが違うものであるという判別はできても,大小関係やスケールが測れるものではない．例えば，次に示すようなTシャツのデータセットがあるとする．

In [1]:
import pandas as pd

tshirt_column = ['price', 'color']
df = pd.DataFrame([
    [1000, 'red'],
    [1500, 'blue'],
    [2300, 'yellow'],
    [500, 'blue']
], columns=tshirt_column)
df

,price,color
0,1000,red
1,1500,blue
2,2300,yellow
3,500,blue


price(値段)は量的変数(比尺度)なので，特に変数変換を行わずとも回帰分析であったり，機械学習であったりに利用することが可能だ．

しかし，color(色)はどうだろうか．この文字列データのままだと人には理解しやすいが，数式・数理モデルに変数として組み込むことはできない．何かしらの数値に変換する必要がある．

このとき用いる変数がダミー変数である．ダミー変数は先述の通り，それぞれの名義尺度の値をバイナリ値で表現する．今回はone-hot vectorに変換する手法をpandasとsklearnの2つの方法で実装する．

In [2]:
# 1. sklearnのOneHotEncoderを用いる方法
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

X=df[['price', 'color']].values

ctf = ColumnTransformer([
    ('nothing', 'passthrough', [0]),
    ('onrhot', OneHotEncoder(), [1])
])

ctf.fit_transform(X)

array([[1000, 0.0, 1.0, 0.0],
       [1500, 1.0, 0.0, 0.0],
       [2300, 0.0, 0.0, 1.0],
       [500, 1.0, 0.0, 0.0]], dtype=object)

In [3]:
# 2. pandasのget_dummiesメソッドを用いる方法
pd.get_dummies(df)

,price,color_blue,color_red,color_yellow
0,1000,0,1,0
1,1500,1,0,0
2,2300,0,0,1
3,500,1,0,0


単にダミー変数への変換を行う(かつDataFrame)のであれば，get_dummiesを用いた方が簡潔だろう．sklearnの方法はColumnTransformerを経由することでOne-hot encoding以外の変換手法も組み合わせることができる，という点で柔軟性がある．

## ダミー変数の次元
---

先ほどの例では，取りうる値が「赤」「青」「黄」3種類(3水準)であった．これに対してOne-hot encodingを行い，3次元のバイナリ値へ変換を行った．

考慮すべきは，この次元数が3ではなく2でも良いということである．例えば，「赤」に関する属性color_redを仮に削除したとしても，そのTシャツが赤色であるかどうかは他の「青」「黄」の属性color_blue, color_yellowを見ればわかる．color_blue, color_yellow=0であれば，color_red=1なのだから．

とりわけダミー変数へ変換する名義特徴量の値に強い相関が予想される場合に，このような次元の削減を行うことがある．具体的には，多重共線性が及ぼす問題に対処するためである．

多重共線性の例として，重回帰モデルを作成する例を考える．以下のように，定数項$\alpha$と誤差項$\epsilon$に加えて，2種類の変数$\beta_1$と$\beta_2$があるとする．説明変数は$\mathbf{x}$で，目的変数は$y$とする．

\begin{eqnarray}
y = \alpha + \beta_1x_1 + \beta_2x_2 + \epsilon
\end{eqnarray}

このとき，変数$x_1$と$x_2$に強い相関が認められる場合は，双方の係数$\beta_1$と$\beta_2$が過小・過大に評価される恐れがあり，正確な分析ができなくなる．

例えば，変数$x_1$と$x_2$に強い正の相関があるとする．導出された重回帰モデルは変数$x_1$のみで十分目的変数を推測できると判断して，本来目的変数に大きな影響を与えるはずの$x_2$の係数$\beta_2$を過小評価するかもしれない．

このような強い相関に基づく悪影響を低減するために，あえてダミー変数をひとつ減らすといった工夫が取られる．実装では，パラメータを指定することでこれが可能だ．

In [4]:
# 次元数を(水準数-1)に減らす(scikit-learn)
X=df[['price', 'color']].values

ohe = OneHotEncoder(categories='auto', drop='first')

ctf = ColumnTransformer([
    ('nothing', 'passthrough', [0]),
    ('onrhot', ohe, [1])
])

ctf.fit_transform(X)

array([[1000, 1.0, 0.0],
       [1500, 0.0, 0.0],
       [2300, 0.0, 1.0],
       [500, 0.0, 0.0]], dtype=object)

In [5]:
# 次元数を(水準数-1)に減らす(pandas)
pd.get_dummies(df, drop_first=True)

,price,color_red,color_yellow
0,1000,1,0
1,1500,0,0
2,2300,0,1
3,500,0,0


ただ，scikit-learnのOneHotEncoderのドキュメントには次のような記述がみられる．

> However, dropping one category breaks the symmetry of the original representation and can therefore induce a bias in downstream models, for instance for penalized linear classification or regression models. 

(Google翻訳)
> ただし、1つのカテゴリを削除すると、元の表現の対称性が失われるため、たとえばペナルティ付き線形分類または回帰モデルの場合、ダウンストリームモデルにバイアスが生じる可能性があります。

例えば，係数にペナルティを課すL2ノルム等を用いる場合は，そもそも扱う変数を削除することによってペナルティを課すはずの係数も消えてしまうということになる．こうした状況がモデルの推測結果に悪影響を与える可能性もあるということだろう．

分析対象・分析方法によって柔軟な対応が求められそうだ．

## 参考文献
---

Sebastian Raschka, Vahid Mirjalili, [第3版]Python機械学習プログラミング, インプレス，2020.

菅由紀子ら, 最短突破 データサイエンティスト検定(リテラシーレベル)公式リファレンスブック, 技術評論社，2021.

[sklearn.preprocessing.OneHotEncoder，scikit-learn 1.0.2 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)

[sklearn.compose.ColumnTransformer，scikit-learn 1.0.2 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html)

[pandas.get_dummies，pandas 1.4.1 documentation](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html)